## Overliew

In [1]:
%matplotlib widget
import matplotlib.pylab as plt
import seaborn as sns

import numpy as np

import pandas as pd

import re

from tqdm import tqdm
import glob

import geopy
import rasterio
from owslib.wms import WebMapService
import folium

## Floading maps

https://gis.nst.dk/server/rest/services/ekstern

ADD SOME INFO

In [23]:
df_flood_maps = pd.read_csv('Plantrin_2.csv')
df_flood_maps.head(20)

,zone,url
0,Aabenraa,https://gis.nst.dk/server/services/ekstern/Aab...
1,Esbjerg,https://gis.nst.dk/server/services/ekstern/Esb...
2,Fredericia,https://gis.nst.dk/server/services/ekstern/Fre...
3,Holstebro,https://gis.nst.dk/server/services/ekstern/Hol...
4,Juelsminde,https://gis.nst.dk/server/services/ekstern/Jue...
5,Kobenhavn_NS,https://gis.nst.dk/server/services/ekstern/Kob...
6,Kobenhavn_Nord,https://gis.nst.dk/server/services/ekstern/Koe...
7,Kobenhavn_Syd,https://gis.nst.dk/server/services/ekstern/Koe...
8,Kolding,https://gis.nst.dk/server/services/ekstern/Kol...
9,Korsoer,https://gis.nst.dk/server/services/ekstern/Kor...


In [24]:
# Quick fix. These two are not in WMS format. Needs to be fixed later
df_flood_maps.drop([6,7], inplace=True);

With the owmlib we can check what data and methods are available

(to get the available data formats: wms.getOperationByName('GetMap').formatOptions)

Here I'm just checking the number of the layers we are interested in

In [25]:
wms = WebMapService('https://gis.nst.dk/server/services/ekstern/Kobenhavn_NS_Plantrin2/MapServer/WmsServer')

for layer_nb in wms.contents:
    print('%s %40s %s'%(layer_nb,wms[layer_nb].title,wms[layer_nb].crsOptions))

1                         Risikoområde_KHS ['EPSG:25832', 'EPSG:4326']
4 Kulturarv_Seværdige_Fortidsminder_punkter_KHS ['EPSG:25832', 'EPSG:4326']
5 Kulturarv_Seværdige_Fortidsminder_areal_KHS ['EPSG:25832', 'EPSG:4326']
6                     Kulturarv_Kirker_KHS ['EPSG:25832', 'EPSG:4326']
7          Kulturarv_Fredede_Bygninger_KHS ['EPSG:25832', 'EPSG:4326']
8      Kulturarv_Fortidsminder_punkter_KHS ['EPSG:25832', 'EPSG:4326']
9        Kulturarv_Fortidsminder_areal_KHS ['EPSG:25832', 'EPSG:4326']
10                   Bes_sten_jorddiger_KHS ['EPSG:25832', 'EPSG:4326']
11                   Begravelsesomraade_KHS ['EPSG:25832', 'EPSG:4326']
13                     Vandvaerksboring_KHS ['EPSG:25832', 'EPSG:4326']
14 Miljoeinteresser_Vandforsyningsboring_KHS ['EPSG:25832', 'EPSG:4326']
15          Miljoeinteresser_Natura2000_KHS ['EPSG:25832', 'EPSG:4326']
16     Miljoeinteresser_Beskyttet_natur_KHS ['EPSG:25832', 'EPSG:4326']
17 Miljoeinteresser_Aktiv_vandindvinding_KHS ['EPSG:25832', 'E

In [26]:
def check_layer_number(url,layer_name):
    wms = WebMapService(url+'/WmsServer')
    layer_titles = [wms[layer_nb].title for layer_nb in wms.contents]
    layer_number = [layer_nb for layer_nb in wms.contents if re.match(layer_name,wms[layer_nb].title) ]
    return layer_number
   
df_flood_maps['Udbredelse_MT20_2019'] = df_flood_maps.url.apply(lambda x: check_layer_number(x,'Udbredelse_\w+_MT20_2019'))

In [27]:
df_flood_maps.head(20)

,zone,url,Udbredelse_MT20_2019
0,Aabenraa,https://gis.nst.dk/server/services/ekstern/Aab...,[99]
1,Esbjerg,https://gis.nst.dk/server/services/ekstern/Esb...,[101]
2,Fredericia,https://gis.nst.dk/server/services/ekstern/Fre...,[96]
3,Holstebro,https://gis.nst.dk/server/services/ekstern/Hol...,[79]
4,Juelsminde,https://gis.nst.dk/server/services/ekstern/Jue...,[99]
5,Kobenhavn_NS,https://gis.nst.dk/server/services/ekstern/Kob...,"[101, 216]"
8,Kolding,https://gis.nst.dk/server/services/ekstern/Kol...,[109]
9,Korsoer,https://gis.nst.dk/server/services/ekstern/Kor...,[100]
10,Koege,https://gis.nst.dk/server/services/ekstern/Koe...,[99]
11,Nyborg,https://gis.nst.dk/server/services/ekstern/Nyb...,[101]


## Save all layers

In [28]:
def save_file(url,layer,size,crs,dir):
    
    wms = WebMapService(url+'/WmsServer')
    srs = wms[layer].crsOptions
    bbox = wms[layer].boundingBoxWGS84 
    output_name = wms[layer].title
    
    if crs not in srs:
        print('CRS not available (available options:)',srs)
        pass
        
    # Get the image from the server
    img = wms.getmap(layers=[layer],
                     styles=['default'],
                     srs=crs,
                     bbox=bbox,
                     size=size,
                     transparent=True,
                     format='image/png8'
                    )
    
    # Clumsy, but the img object is not an array, so it needs to be saved to a file
    with open(dir+output_name+'.png', 'wb') as out:
        out.write(img.read())
       
    # Re-open the same image in rasterio to georeference it
    # for tiff save only one band
    img_array = rasterio.open(dir+output_name+'.png').read(1).squeeze() 
    img_shape = img_array.shape
    dst_transform = rasterio.transform.from_bounds(west = bbox[0],
                                                   south = bbox[1],
                                                   east = bbox[2],
                                                   north = bbox[3],
                                                   width=img_shape[1], height=img_shape[0])
    # Write it out to a file.
    with rasterio.open(dir+output_name+'_georef.png', 'w', driver='GTiff',
                                  width=img_shape[1], height=img_shape[0],
                                  count=1, dtype=np.uint8, nodata=0,
                                  transform=dst_transform, crs=crs) as dst:
        dst.write(img_array.astype(np.uint8), indexes=1)

In [30]:
for idx,row in df_flood_maps.iterrows():
    for layer in row.Udbredelse_MT20_2019:
        print(row.url,layer)
        save_file(row.url,
                  layer=layer,
                  size=(1000,1000),
                  crs='EPSG:4326',
                  dir='Udbredelse_MT20_2019_HR/')

https://gis.nst.dk/server/services/ekstern/Aabenraa_Plantrin2/MapServer 99


ServiceException: Parameter 'width' contains unacceptable value.

## Plot all with folium

In [ ]:
m = folium.Map([55.39594, 10.38831])

group = folium.FeatureGroup(name='Test')


#for file in glob.glob('Udbredelse_MT20_2019/*geo*'):
for file in glob.glob('Udbredelse_MT20_2019_HR/*geo*'):
    
    print(file)
    # Add layer
    rio_img = rasterio.open(file)
    bbox = rio_img.bounds

    group.add_child(folium.raster_layers.ImageOverlay(
                    name=file,
                    image = rio_img.read().squeeze(),
                    #[[lat_min, lon_min], [lat_max, lon_max]]
                    bounds = [[bbox.bottom,bbox.left],[bbox.top,bbox.right]],
                    opacity=0.8,
                    show=True,
                    overlay=True,
                    interactive=True,
                    )
                   )
                    
m.add_child(group)
                    
folium.LayerControl().add_to(m)

## Add a point
folium.Marker([55.39594, 10.38831], popup='this is a marker').add_to(m)

m